In [1]:
from importlib import reload
import traceback
import json
import pygame as pg
import os

import enemy  # You may need to re-import the base module
import world
import constants as c
import turret_data as td
import enemy_data as ed
import turret
import button
import save_load_manager
import stage_data
reload(stage_data)
reload(save_load_manager)
reload(enemy)
reload(world)
reload(c)
reload(turret)
reload(button)
reload(td)
reload(ed)
from save_load_manager import SaveLoadClass as slc
from enemy import Enemy
from world import World
from turret import Turret
from button import Button
from stage_data import Stages

class Game:
    def __init__(self):
        
        # Initiate game
        pg.init()
        pg.display.init()
        self.clock = pg.time.Clock()
        self.screen = pg.display.set_mode((c.SCREEN_WIDTH + c.SIDE_PANEL,c.SCREEN_HEIGHT))
        pg.display.set_caption('Tower Defence')
        self.run = False
        # Load Fonts
        self.small_font = pg.font.SysFont('Consolas', 15, bold=False)
        self.text_font = pg.font.SysFont('Consolas', 24, bold=True)
        self.large_font = pg.font.SysFont('Consolas', 36) 

        # Game Variables
        self.stages = Stages()
        self.stage_data = self.stages.stages[0]
        self.game_over = False
        self.game_outcome = 0 # -1 is a loss 1 is a win
        self.last_enemy_spawn = pg.time.get_ticks()
        self.placing_turrets = False
        self.selected_turret = None
        self.level_started = False
        

        ######## Load images:
        self.map_image = pg.image.load('Levels/level.png').convert_alpha()
        
        # Enemies  ########## Add new enemy types here
        self.enemy_data = {
            'regular':ed.ENEMY_DATA_REG,
            'dash': ed.ENEMY_DATA_DASH,
            'armored': ed.ENEMY_DATA_ARMORED,
            'constant':ed.ENEMY_DATA_CONSTANT,
            'speed': ed.ENEMY_DATA_SPEED,
            'strong': ed.ENEMY_DATA_STRONG,
        }
        
            # Turrets
        self.change_target_image = self.text_font.render('Change Target Type', True, '#000000')
        self.turret_types = {
            'regular':td.TURRET_DATA,
            'strong':td.STRONG_TURRET_DATA,
            'fast':td.FAST_TURRET_DATA,
            'emp':td.EMP_TURRET_DATA,
            'other':td.OTHER_TURRET_DATA,
            'another':td.ANOTHER_TURRET_DATA
        }
        
            # Button Images
        self.buy_turret_image = pg.image.load('Assets/Buttons/buy_turret.png').convert_alpha()
        self.cancel_image = pg.image.load('Assets/Buttons/cancel.png').convert_alpha()
        self.upgrade_turret_image = pg.image.load('Assets/Buttons/upgrade_turret.png').convert_alpha()
        self.begin_image = pg.image.load('Assets/Buttons/begin.png').convert_alpha()
        self.restart_image = pg.image.load('Assets/Buttons/restart.png').convert_alpha()
        self.fast_forward_image = pg.image.load('Assets/Buttons/fast_forward.png').convert_alpha()
            # GUI Images
        self.coin_image = pg.image.load('Assets/Gui/coin.png').convert_alpha()
        self.heart_image = pg.image.load('Assets/Gui/heart.png').convert_alpha()
            # Turret Images
        self.turret_images = {}
        for turret_type in self.turret_types:
            self.turret_images[turret_type] = pg.image.load(self.turret_types[turret_type]['image'])
            # Rescale the images
            if turret_type == 'emp':
                self.turret_images[turret_type] = pg.transform.smoothscale_by(self.turret_images[turret_type], 0.6)
            else:
                self.turret_images[turret_type] = pg.transform.smoothscale_by(self.turret_images[turret_type], 0.85)
        self.change_turret(td.TURRET_DATA)

        #load json data for level
        with open('Levels/level.tmj') as file:
            self.world_data = json.load(file)

        # Create world
        self.world = World(self.world_data, self.map_image, self.stage_data)
        self.world.process_data()
        self.world.process_enemies()

        # Create Groups
        self.enemy_group = pg.sprite.Group() 
        self.turret_group = pg.sprite.Group()

        # Create Buttons
        self.buy_turret_button = Button(c.SCREEN_WIDTH + 30, 120, self.buy_turret_image)
        self.cancel_button = Button(c.SCREEN_WIDTH + 50, 180, self.cancel_image)
        self.upgrade_button = Button(c.SCREEN_WIDTH + 5, 180, self.upgrade_turret_image)
        self.begin_button = Button(c.SCREEN_WIDTH + 60, 300, self.begin_image)
        self.restart_button = Button(310, 300, self.restart_image)
        self.fast_forward_button = Button(c.SCREEN_WIDTH + 60, 300, self.fast_forward_image, single_click = False)
            # Turret Buttons
        self.turret_buttons = {}
        for turret_type in self.turret_types:
            self.turret_buttons[turret_type] = Button(self.turret_types[turret_type]['button_x'], 
                                                      self.turret_types[turret_type]['button_y'], 
                                                      self.turret_images[turret_type])
        self.target_type_button = Button(c.SCREEN_WIDTH + 5, 600, self.change_target_image)
    
    def __del__(self):
        pg.quit()
        

    def play(self, turret_group = None):
        if not turret_group == None:
            self.turret_group = turret_group
        self.run = True
        while self.run:
            self.clock.tick(c.FPS)
            self.screen.fill('grey100')
    
            ##################
            # Update
            ##################
            if not self.game_over:
                self.game_over = self.check_game_over()
                    
                self.enemy_group.update(self.world)
                self.turret_group.update(self.enemy_group, self.world)
                self.cur_pos = pg.mouse.get_pos()
                if self.selected_turret:
                    self.selected_turret.selected = True 
            
            ##################
            # Draw
            ##################
            self.world.draw(self.screen)
            self.enemy_group.draw(self.screen)
            self.draw_stats()
            for self.turret in self.turret_group:
                self.turret.draw(self.screen)
                
            if not self.game_over:
                self.draw_map()              
            else:
                self.draw_game_over()
                    
            self.event_handler()
            # update display
            pg.display.flip()
        
        
    def event_handler(self):
        # Event Handler
        for event in pg.event.get():
            if event.type == pg.QUIT:
                self.run = False

            if event.type == pg.MOUSEBUTTONDOWN and event.button == 1:
                if self.cur_pos[0] < c.SCREEN_WIDTH and self.cur_pos[1] < c.SCREEN_HEIGHT:
                    self.clear_selection()
                    if self.placing_turrets:
                        self.create_turret()
                    else:
                        self.selected_turret = self.select_turret()

    def draw_game_over(self):
        pg.draw.rect(self.screen, 'blue', (200,200,400,200), border_radius = 30)
        if self.game_outcome == -1:
            self.draw_font(f'GAME OVER', self.large_font, 310, 250, image_color = '#000000')
        if self.game_outcome == 1:
            self.draw_font(f'YOU WON!!', self.large_font, 310, 250, image_color = '#000000')
        
        if self.restart_button.draw(self.screen):
            self.restart()
            
    def draw_map(self):
        if not self.level_started:
            if self.begin_button.draw(self.screen):
                self.level_started = True
        else:
            self.world.game_speed = 1
            if self.fast_forward_button.draw(self.screen):
                self.world.game_speed = 2
            # Spawn Enemies
            if pg.time.get_ticks() - self.last_enemy_spawn > (c.SPAWN_COOLDOWN/self.world.game_speed):
                if self.world.spawned_enemies < len(self.world.enemy_list):
                    self.add_enemy()
        if self.world.check_level_complete():
            if not self.check_game_over():
                self.finish_level()
            
        if self.buy_turret_button.draw(self.screen):
            self.placing_turrets = True
            
        if self.placing_turrets: # Mouse turret/ Cancel button
            self.draw_placing_turret()

        if self.selected_turret: 
            self.draw_selected_turret()

    def change_turret(self, turret_data):
        self.turret_data = turret_data
        self.turret_image = self.turret_images[turret_data['turret_type']]
        
        reach = self.turret_data['level'][0].get('reach')
        self.range_image = pg.Surface((reach*2, reach*2))
        self.range_image.fill((0,0,0))
        self.range_image.set_colorkey((0,0,0))
        pg.draw.circle(self.range_image, ('grey100'), (reach, reach), reach)
        self.range_image.set_alpha(100)
        self.range_rect = self.range_image.get_rect()

    def draw_placing_turret(self):
        self.selected_turret = None
        cursor_rect = self.turret_image.get_rect()
        cursor_rect.center = self.cur_pos
        
        if self.cur_pos[0] <= c.SCREEN_WIDTH:
            self.screen.blit(self.turret_image, cursor_rect)
            self.range_rect.center = cursor_rect.center
            self.screen.blit(self.range_image, self.range_rect)
        self.draw_font(f"${self.turret_data['cost']}", self.text_font, c.SCREEN_WIDTH + 200, 130, image_color = '#000000')
        self.draw_font(f"{self.turret_data['description']}", self.small_font, c.SCREEN_WIDTH + 1, 700, image_color = '#000000')
        # Draw Cancel Button
        if self.cancel_button.draw(self.screen):
            self.placing_turrets = False
        
        for turret_type in self.turret_types:
            if self.turret_buttons[turret_type].draw(self.screen):
                self.change_turret(self.turret_types[turret_type])
            
    
    def draw_selected_turret(self):
        self.placing_turrets = False
        self.draw_font(f"{self.selected_turret.turret_data['description']}", self.small_font, c.SCREEN_WIDTH + 1, 700, image_color = '#000000')
        self.draw_font(f"Range: {self.selected_turret.turret_data['level'][self.selected_turret.level-1].get('reach')}", 
                       self.small_font, c.SCREEN_WIDTH + 1, 640, image_color = '#000000')
        self.draw_font(f"Damage: {self.selected_turret.turret_data['level'][self.selected_turret.level-1].get('damage')}", 
                       self.small_font, c.SCREEN_WIDTH + 1, 670, image_color = '#000000')
        self.draw_font(f"Cooldown: {self.selected_turret.turret_data['level'][self.selected_turret.level-1].get('cooldown')}", 
                       self.small_font, c.SCREEN_WIDTH + 1, 655, image_color = '#000000')
        self.draw_font(f"Target Type: {self.selected_turret.target_type}", 
                       self.small_font, c.SCREEN_WIDTH + 1, 685, image_color = '#000000')
        
        if self.target_type_button.draw(self.screen):
            self.selected_turret.change_target_type()
            
        if self.selected_turret.level < self.selected_turret.max_level: 
            self.draw_font(f'${self.selected_turret.upgrade_cost}', self.text_font, c.SCREEN_WIDTH + 220, 200, image_color = '#000000')
            if self.upgrade_button.draw(self.screen):
                if self.world.money >= self.selected_turret.upgrade_cost:
                    self.world.money -= self.selected_turret.upgrade_cost
                    self.selected_turret.upgrade()

        
    def draw_stats(self, draw_money = True):
        self.draw_font(f'HP {self.world.health}', self.text_font, c.SCREEN_WIDTH + 40, 5, image_color = '#000000')
        self.screen.blit(self.heart_image, (c.SCREEN_WIDTH + 5, 0))
        if draw_money:
            self.draw_font(f'{self.world.money}', self.text_font, c.SCREEN_WIDTH + 42, 37, image_color = '#000000')
            self.screen.blit(self.coin_image, (c.SCREEN_WIDTH + 5, 30))
        self.draw_font(f'Level: {self.world.level}', self.text_font, c.SCREEN_WIDTH + 5, 70, image_color = '#000000')
        
    
    def check_game_over(self):
        if self.world.health <= 0: # Lost
            self.game_outcome = -1
            return True
        elif self.world.level > len(self.world.level_spawn_data):
            self.game_outcome = 1
            return True
        return False
            
            
    def finish_level(self):
        self.last_enemy_spawn = pg.time.get_ticks()
        self.world.process_enemies()
        self.level_started = False
        
    def add_enemy(self):
        enemy_level = self.world.enemy_list[self.world.spawned_enemies]
        enemy = Enemy(enemy_level, self.world.waypoints, self.enemy_data[self.world.enemy_type_list[self.world.spawned_enemies]])
        self.enemy_group.add(enemy)
        self.world.spawned_enemies += 1
        self.last_enemy_spawn = pg.time.get_ticks()
        
    def restart(self, reset_turret_group = True):
        self.game_over = False
        self.game_outcome = 0 # -1 is a loss 1 is a win
        self.last_enemy_spawn = pg.time.get_ticks()
        self.placing_turrets = False
        self.selected_turret = None
        self.level_started = False
        
        self.world = World(self.world_data, self.map_image, self.stage_data)
        self.world.process_data()
        self.world.process_enemies()
        # empty groups
        self.enemy_group.empty()
        if reset_turret_group:
            self.turret_group.empty()
        
    def draw_font(self, text, font, x, y, image_color = '#000000'):
        img = font.render(text, True, image_color)
        self.screen.blit(img, (x,y))

    def create_turret(self):
        mouse_tile_x = self.cur_pos[0] // c.TILE_SIZE
        mouse_tile_y = self.cur_pos[1] // c.TILE_SIZE
        mouse_tile_num = mouse_tile_y * c.COLS + mouse_tile_x
        
        if self.world.tile_map[mouse_tile_num] == 7:
            if not (mouse_tile_x, mouse_tile_y) in [(tg.tile_x, tg.tile_y) for tg in self.turret_group]:
                if self.world.money >= self.turret_data['cost']:
                    turret = Turret(mouse_tile_x, mouse_tile_y, self.turret_data)
                    self.turret_group.add(turret)
                    self.world.money -= self.turret_data['cost']
    
    def select_turret(self):
        mouse_tile_x = self.cur_pos[0] // c.TILE_SIZE
        mouse_tile_y = self.cur_pos[1] // c.TILE_SIZE
        for turret in self.turret_group:
            if (mouse_tile_x, mouse_tile_y) == (turret.tile_x, turret.tile_y):
                return turret
    
    def clear_selection(self):
        for turret in self.turret_group:
            turret.selected = False
    
    def main_menu(self):
        self.defense_slc = slc()
        self.offense_slc = slc(file_extension='.atd') # attack tower defense
        # Create menu image options
        self.play_image = self.large_font.render('Play New Game', True, '#000000')
        self.create_defense_image = self.large_font.render('Create Defense', True, '#000000')
        self.load_defense_image = self.large_font.render('Load Defense', True, '#000000')
        self.load_offense_image = self.large_font.render('Load Offense', True, '#000000')
        self.run_sim_image = self.large_font.render('Run Simulation', True, '#000000')
        # Create other images
        self.current_stage_image = self.text_font.render(f"Stage: {self.stage_data['name']}", True, '#000000')
        self.current_defense = self.text_font.render('Defense: None', True, '#000000')
        # Create Menu Buttons
        self.play_button = Button(c.SCREEN_WIDTH + 10, 10, self.play_image)
        self.create_defense_button = Button(c.SCREEN_WIDTH + 10, 110, self.create_defense_image)
        self.load_defense_button = Button(c.SCREEN_WIDTH + 10, 160, self.load_defense_image)
        self.load_offense_button = Button(c.SCREEN_WIDTH + 10, 210, self.load_offense_image)
        self.run_sim_button = Button(c.SCREEN_WIDTH + 10, 260, self.run_sim_image)

        self.run = True
        while self.run:
            ##################
            # Update
            ##################
            self.clock.tick(c.FPS)
            self.screen.fill('grey100')
            self.cur_pos = pg.mouse.get_pos()
            
            ##################
            # Draw
            ##################
            self.world.draw(self.screen)
            self.screen.blit(self.current_stage_image, (c.SCREEN_WIDTH + 5, 400))
            self.screen.blit(self.current_defense, (c.SCREEN_WIDTH + 5, 440))
            
            for self.turret in self.turret_group:
                self.turret.draw(self.screen)   
            
            if self.play_button.draw(self.screen):
                self.turret_group.empty()
                self.play()
            # Create Defense
            elif self.create_defense_button.draw(self.screen):
                self.turret_group = self.make_defense()
                if self.run:
                    self.defense_slc.save_data([turret.to_dict() for turret in self.turret_group])
            # Load Defense
            elif self.load_defense_button.draw(self.screen):
                file_to_load = self.load_defense()
                if file_to_load: # If you clicked on a file to load
                    self.turret_group.empty()
                    loaded_turret_data = self.defense_slc.load_data(file_to_load)
                    for turret_data in loaded_turret_data:
                        print(turret_data)
                        turret = Turret(turret_data['tile_x'], turret_data['tile_y'], 
                                       self.turret_types[turret_data['turret_type']], 
                                        turret_data['level'], turret_data['target_type'])
                        self.turret_group.add(turret)
            # Load Offense
            elif self.load_offense_button.draw(self.screen):
                self.stage_data = self.load_offense()
                self.current_stage_image = self.text_font.render(f"Stage: {self.stage_data['name']}", True, '#000000')
                self.restart(reset_turret_group = False)
            elif self.run_sim_button.draw(self.screen):
                self.run_sim()
            
            # Event Handler
            self.event_handler()
            pg.display.flip()

  
    def load_defense(self):
        # Path to the saves directory
        saves_directory = "./Saves"
        
        # Store buttons for each save file
        file_buttons = []
    
        # Loop through the save files and create buttons for each
        j = 0 # Make it so there is no spaces
        for i, file_name in enumerate(os.listdir(saves_directory)):
            if file_name[-3:] == '.td':
                # Create the file name image
                file_image = self.small_font.render(file_name, True, '#000000')
                # Create a button for the file (stacking vertically)
                file_button = Button(105, 105 + (i - j + 1) * 15, file_image)  # Adjust spacing as needed
                file_buttons.append((file_button, file_name))
            else:
                j += 1
    
        while self.run:
            # Draw the game world and UI background
            self.world.draw(self.screen)
            pg.draw.rect(self.screen, '#aaaaaa', (100, 100, 500, 500), border_radius=30)
    
            # Draw each button and check if it is pressed
            for file_button, file_name in file_buttons:
                if file_button.draw(self.screen):  # If button is pressed
                    self.current_defense = self.text_font.render(f'Defense: {file_name}', True, '#000000')
                    return os.path.join(saves_directory, file_name)  # Return full file path
    
            # Update the display
            pg.display.flip()
            self.event_handler()
            
    def load_offense(self):
        # Reload the stage data
        self.stages = Stages()
        
        # Store buttons for each save file
        stage_buttons = []
    
        # Loop through the save files and create buttons for each
        for i, stage in enumerate(self.stages.stages):
            # Create the file name image
            stage_image = self.small_font.render(stage['name'], True, '#000000')
            # Create a button for the file (stacking vertically)
            stage_button = Button(105, 105 + (i + 1) * 15, stage_image)  # Adjust spacing as needed
            stage_buttons.append((stage_button, stage))
    
        while self.run:
            # Draw the game world and UI background
            self.world.draw(self.screen)
            pg.draw.rect(self.screen, '#aaaaaa', (100, 100, 500, 500), border_radius=30)
    
            # Draw each button and check if it is pressed
            for stage_button, stage in stage_buttons:
                if stage_button.draw(self.screen):  # If button is pressed
                    return stage
    
            # Update the display
            pg.display.flip()
            self.event_handler()
    
    def make_defense(self):
        self.done_image = self.large_font.render('DONE', True, '#000000')
        self.done_button = Button(c.SCREEN_WIDTH + 60, 300, self.done_image)
        while self.run:
            self.clock.tick(c.FPS)
            self.screen.fill('grey100')
    
            ##################
            # Update
            ##################
            self.cur_pos = pg.mouse.get_pos()
            if self.selected_turret:
                self.selected_turret.selected = True 
            
            ##################
            # Draw
            ##################
            self.world.draw(self.screen)
            self.draw_stats()
            
            for self.turret in self.turret_group:
                self.turret.draw(self.screen)
            
            # Event Handler
            self.event_handler()
            if self.done_button.draw(self.screen):
                break
            if self.buy_turret_button.draw(self.screen):
                self.placing_turrets = True
                
            if self.selected_turret:
                self.draw_selected_turret()
                
            if self.placing_turrets: # Mouse turret/ Cancel button
                self.draw_placing_turret()
            # update display
            pg.display.flip()
        return self.turret_group
    
    def run_sim(self, world_speed = 1):
        self.slow_image = self.text_font.render("Normal (1X) speed", True, '#000000')
        self.fast_image = self.text_font.render("Fast (2X) speed", True, '#000000')
        self.faster_image = self.text_font.render('Faster (4X) speed', True, '#000000')
        self.fastest_image = self.text_font.render('Faster (8X) speed', True, '#000000')

        self.slow_button = Button(c.SCREEN_WIDTH + 5, 250, self.slow_image)
        self.fast_button = Button(c.SCREEN_WIDTH + 5, 300, self.fast_image)
        self.faster_button = Button(c.SCREEN_WIDTH + 5, 350, self.faster_image)
        self.fastest_button = Button(c.SCREEN_WIDTH + 5, 400, self.fastest_image)

        if not self.turret_group:
            print('You have no turrets, will not run simulation.')
            return
        
        self.run = True
        while self.run:
            self.clock.tick(c.FPS)
            self.screen.fill('grey100')
    
            ##################
            # Update
            ##################
            if not self.game_over:
                self.game_over = self.check_game_over()
                    
                self.enemy_group.update(self.world)
                self.turret_group.update(self.enemy_group, self.world)
                self.cur_pos = pg.mouse.get_pos()
            
            ##################
            # Draw
            ##################
            self.world.draw(self.screen)
            self.enemy_group.draw(self.screen)
            self.draw_stats(draw_money = False)
            for self.turret in self.turret_group:
                self.turret.draw(self.screen)
                
            if not self.game_over:
                if not self.level_started:
                    if self.begin_button.draw(self.screen):
                        self.level_started = True
                else:
                    if self.slow_button.draw(self.screen):
                        self.world.game_speed = 1
                    elif self.fast_button.draw(self.screen):
                        self.world.game_speed = 2
                    elif self.faster_button.draw(self.screen):
                        self.world.game_speed = 4
                    elif self.fastest_button.draw(self.screen):
                        self.world.game_speed = 8
                    # Spawn Enemies
                    if pg.time.get_ticks() - self.last_enemy_spawn > (c.SPAWN_COOLDOWN/self.world.game_speed):
                        if self.world.spawned_enemies < len(self.world.enemy_list):
                            self.add_enemy()
                if self.world.check_level_complete():
                    if not self.check_game_over():
                        self.finish_level()            
            else:
                #self.draw_game_over()
                pg.draw.rect(self.screen, 'blue', (200,200,400,200), border_radius = 30)
                if self.game_outcome == -1:
                    self.draw_font(f'GAME OVER', self.large_font, 310, 250, image_color = '#000000')
                if self.game_outcome == 1:
                    self.draw_font(f'YOU WON!!', self.large_font, 310, 250, image_color = '#000000')
                
                if self.restart_button.draw(self.screen):
                    
                    self.restart(reset_turret_group = False)
                    return
                    
            self.event_handler()
            # update display
            pg.display.flip()
        

try:
    game = Game()
    game.main_menu()
    del game
except Exception as e: 
    print(e)
    pg.quit()
    print(traceback.format_exc())
    


pygame 2.1.3 (SDL 2.0.22, Python 3.11.7)
Hello from the pygame community. https://www.pygame.org/contribute.html
[{'name': 'Stage Data 1', 'spawn_data': [{'regular': [15]}, {'dash': [20]}, {'strong': [20]}, {'regular': [0, 5], 'speed': [25]}, {'constant': [0, 10]}, {'armored': [30]}, {'regular': [15], 'dash': [15], 'speed': [15], 'constant': [15], 'armored': [15], 'strong': [15]}, {'regular': [0, 5], 'dash': [0, 5], 'speed': [0, 5], 'constant': [0, 5], 'armored': [0, 5], 'strong': [0, 5]}, {'regular': [0, 0, 10], 'dash': [0, 0, 15]}, {'speed': [0, 0, 15], 'constant': [0, 0, 10]}, {'armored': [0, 0, 0, 10], 'strong': [0, 0, 0, 10]}], 'level_reward': 50, 'money': 1200, 'health': 50}, {'name': 'Stage Data Balance', 'spawn_data': [{'regular': [20]}, {'regular': [0, 20]}, {'regular': [0, 0, 20]}, {'regular': [0, 0, 0, 20]}], 'level_reward': 50, 'money': 10000, 'health': 20}]
{'tile_x': 12, 'tile_y': 5, 'turret_type': 'regular', 'level': 2, 'target_type': 'default'}
{'tile_x': 11, 'tile_y': 

In [9]:
# Todo
# Tower Types
    # ---High damage slow fire
    # ---Quick fire low damage
    # ---EMP Damage all
    # Range
    # Other 
    # Splash? 
    # Slow/stop Down Enemies? 
# Enemy Types
    # ---Dash: High Damage Low Firerate
    # Healing?
    # ---X number to kill
    # ---Armored Done
    # ---Speed
    # ---Dash
    # Blinking Target?
    # Splitting (Make more when killed)?
    # Combinations?
# Landscape own path
    # Path finding
    # Placable landscape things
# Asthetics:
    # --- Range of tower when placing
    # Turret Icons, Icon for end (castle)
    # Greyed Out buttons (upgrade, buy turret, fully upgraded)
    # View Next Wave
    # Done Button
    # --- Change Targeting Mechanics
# Gameplay Mechanics
    # Restart Button
    # Sell Towers
# Game play
    # (Kaden) Create Defense (Start with a given amount of money and make your defense) 
        # 
    # Run attack
        # Start with turret placements and run through the waves
    # Create waves (Kody)
        # Create a enemy wave pattern with a given amount of money
    # Create Main Menu:
        # New game (Standard waves easy, medium hard?)
        # Create a defense, then save
        # Load in a defense
        # Create enemy wave
        # Run fast game with no loaded defense and selected wave 
    
# Balancing!!!
# Turret class???/ enemy class (I don't think this needs to be done)


In [7]:
reload(td)
turret_types = {
            'regular':td.TURRET_DATA,
            'strong':td.STRONG_TURRET_DATA,
            'fast':td.FAST_TURRET_DATA,
            'emp':td.EMP_TURRET_DATA,
            'other':td.OTHER_TURRET_DATA,
            'another':td.ANOTHER_TURRET_DATA
        }
for tt in turret_types:
    for i,l in enumerate(turret_types[tt]['level']):
        print(f"{tt} {i+1}: {1000*l['damage']/(120 + l['cooldown']):.1f} dam/s")
    print()

regular 1: 3.1 dam/s
regular 2: 7.6 dam/s
regular 3: 14.7 dam/s
regular 4: 24.4 dam/s

slow 1: 3.2 dam/s
slow 2: 7.6 dam/s
slow 3: 14.5 dam/s
slow 4: 31.2 dam/s

fast 1: 3.2 dam/s
fast 2: 7.1 dam/s
fast 3: 14.8 dam/s
fast 4: 30.0 dam/s

emp 1: 0.7 dam/s
emp 2: 1.6 dam/s
emp 3: 2.7 dam/s
emp 4: 5.4 dam/s

other 1: 0.7 dam/s
other 2: 2.3 dam/s
other 3: 3.6 dam/s
other 4: 5.4 dam/s

another 1: 0.7 dam/s
another 2: 2.3 dam/s
another 3: 3.6 dam/s
another 4: 5.4 dam/s



In [13]:
import easygui
dir(easygui)
#easygui.textbox()
print(easygui.enterbox('Input file name: ', title='AHHHHHH', default='your_name'))

None


In [13]:
help(pg.transform.smoothscale_by)
help(pg.transform.set_smoothscale_backend)

Help on built-in function smoothscale_by in module pygame.transform:

smoothscale_by(...)
    smoothscale_by(surface, factor, dest_surface=None) -> Surface
    resize to new resolution, using scalar(s)

Help on built-in function set_smoothscale_backend in module pygame.transform:

set_smoothscale_backend(...)
    set_smoothscale_backend(backend) -> None
    set smoothscale filter version to one of: 'GENERIC', 'MMX', or 'SSE'

